In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import IPython
import platform
import scipy as sp
import seaborn as sns

О снижении размерности: https://studme.org/93348/statistika/snizhenie_razmernosti_priznakovogo_prostranstva

ОСНОВНОЕ ИЗ СТАТЬИ ВЫШЕ:

Снижение размерности необходимо для повышения эффективности работы некоторой модели (регрессия, классификация). Существует даже термин "проклятие размерности" (curse of dimensionality), характеризующий проблемы работы с высокоразмерными данными.

Неинформативные признаки являются источником дополнительного шума и влияют на точность оценки параметров модели. Кроме того, наборы данных с большим числом признаков могут содержать группы коррелированных переменных. 

Можно выделить два направления в снижении размерности признакового пространства по принципу используемых для этого переменных: отбор признаков из имеющегося исходного набора и формирование новых признаков путем трансформации первоначальных данных.

Отбор имеющихся признаков. Прямое решение задачи создания наилучшей модели связано с перебором всех возможных сочетаний признаков. Как правило, прибегают к прямой или обратной селекции признаков. В процедурах прямого отбора производится последовательное добавление переменных из исходного набора до достижения необходимого качества модели. В алгоритмах последовательной редукции исходного признакового пространства (обратной селекции) производится поэтапное удаление наименее информативных переменных до допустимого снижения информативности модели.
Следует учитывать, что информативность признаков относительна. Отбор должен обеспечить высокую информативность набора признаков, а не суммарную информативность составляющих его переменных. 

Формирование новых. Методы отбора признаков связаны с предположением о непосредственном присутствии необходимых признаков в исходных данных, но это может оказаться неверным. Альтернативный подход к снижению размерности предусматривает преобразование признаков в сокращенный набор новых переменных. Формирование нового признакового пространства предполагает создание новых переменных, которые обычно являются функциями исходных признаков. Эти переменные, непосредственно не наблюдаемые, часто называют скрытыми, или латентными. 

Преобразование исходного признакового пространства удобно для решения задач классификации, так как позволяет обоснованно применять определенные меры близости или различий объектов, такие как евклидово расстояние либо квадрат евклидова расстояния. В регрессионном анализе построение уравнения регрессии на главных компонентах позволяет решить проблему мультиколлинеарности.

АЛГОРИТМ следующий:
- Выделение целевой переменной - и построение модели регрессии/классификации
- Применение для всего набора переменных различных методов снижения размерности. При этом следует обратить внимание на следущие показатели:
    - информативность переменных
    - качество модели
    - количество выделенных переменных
- Информативность перемнных - количество информации об объекте, содержащейся в значениях переменной с точки зрения задачи модели. При исключении перемнных из набора информативность не должна падать ниже отмеченной нормы.
- Итерация метода снижения размерности имеет смысл, если при этом качество модели растет.
- Чем меньше переменных модели (чем проще модель), тем лучше, но (!!!) только при соблюдении условий в предыдущих пунктах ограничений.

ЗАГРУЗКА ДАННЫХ И ПРОСМОТР

In [4]:
data = pd.read_csv('Наборы данных_экз/Наборы данных/Снижение размерности/Cattell\'s 16 Personality Factors/499372_925801_bundle_archive/16PF/data.csv', sep='\t')
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,24,1,100,IN,1,274
49155,4,5,4,4,4,3,5,3,2,3,...,3,4,4,4,22,1,50,ZA,1,584
49156,2,4,2,3,3,3,4,4,2,4,...,3,4,5,4,29,2,85,BE,6,932
49157,4,4,4,4,4,4,4,4,1,3,...,2,4,5,4,48,1,95,CZ,2,1212


Данные содержат результаты опроса по некоторым категориям с вариантами ответов (0-5), а также метаданные каждого опрашиваемого - столбцы age, country, gender. 

Столбец accuracy, выражающий удовлетворенность опрашиваемого результатами опроса. 

Столбец elapsed показывает время, затраченное опрашиваемым, на прохождение опроса.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Columns: 169 entries, A1 to elapsed
dtypes: int64(168), object(1)
memory usage: 63.4+ MB


In [6]:
print(data.nunique())
print(data.shape)

A1             6
A2             6
A3             6
A4             6
A5             6
            ... 
gender         4
accuracy     113
country      159
source         6
elapsed     4587
Length: 169, dtype: int64
(49159, 169)


ПОСТАНОВКА ЗАДАЧИ МОДЕЛИ.

Считаю, что будет рациональным построение модели РЕГРЕССИИ по данному датасету с целевой переменной accuracy.

ПРЕДОБРАБОТКА

Если в данных есть nan, необходимо либо удалять целую строчку, либо заполнить (самым частотным - для категориальным, средним - для числовых).

Попробуем найти nan, и если найдутся, удалить их

In [7]:
def reindexing (ind):
    arr = list(ind)
    res = []
    for i in range(len(arr)):
        res.append(i)
    return res
start_shape = data.shape
data.dropna(inplace=True)
data.index = reindexing(data.index)
end_shape = data.shape
if start_shape==end_shape:
        print('no NaNs in the dataset. Shape: ',end_shape)
else:
    print('NaNs delited. Start shape: ', start_shape, ' End shape: ', end_shape)

NaNs delited. Start shape:  (49159, 169)  End shape:  (49142, 169)


Одна из переменных counrty выражена категориально. Ее значение следует шкалировать.

In [8]:
from sklearn import preprocessing

countries=[]
for i in data.iloc[:,166]:
    if i not in countries:
        countries.append(i)

le = preprocessing.LabelEncoder()
le.fit(countries)
data.country = le.transform(data.country)
print(data)

       A1  A2  A3  A4  A5  A6  A7  A8  A9  A10  ...  P7  P8  P9  P10  age  \
0       1   4   2   3   3   2   3   4   4    3  ...   1   2   5    2   17   
1       4   3   4   3   4   4   4   4   2    2  ...   3   2   2    2   37   
2       3   4   4   4   4   4   4   3   2    2  ...   2   3   3    3   31   
3       4   5   4   4   4   3   3   2   2    2  ...   2   3   4    4   32   
4       4   0   4   4   4   3   5   1   2    4  ...   4   2   3    2   46   
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ..  ..  ..  ...  ...   
49137   1   1   1   1   1   1   1   1   1    1  ...   1   1   1    1   24   
49138   4   5   4   4   4   3   5   3   2    3  ...   3   4   4    4   22   
49139   2   4   2   3   3   3   4   4   2    4  ...   3   4   5    4   29   
49140   4   4   4   4   4   4   4   4   1    3  ...   2   4   5    4   48   
49141   4   5   4   5   5   4   5   3   1    2  ...   1   4   4    5   18   

       gender  accuracy  country  source  elapsed  
0           1        92

ПОСТРОЕНИЕ МОДЕЛИ

Так как в наборе переменных большой процент ординальных переменных (А1 - Р10), думаю, что рационально использование ???? регрессии.

In [9]:
# Разделим выборку на тестовую и тренировочную и проведем стадартизацию данных
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

def split_scale (data):
    # Split
    arr_for_x = []
    for i in range(len(data.columns)):
        arr_for_x.append(i)
    y = arr_for_x.pop(165)
#     y = arr_for_x.pop(168)
    X, y = d= data.iloc[:,arr_for_x],  data.iloc[:,y]
    # Test set will be the 25% taken randomly
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

    # Standarize
    # scaler = preprocessing.MinMaxScaler(feature_range=(0, 100)).fit(X_train)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    # StandardScaler()

#     print(pd.DataFrame(X_train))

    return X_train, X_test, y_train, y_test

split_scale (data)

(array([[-0.56611479, -0.75179025, -1.62742151, ...,  0.85971666,
         -0.83203235, -0.03867379],
        [-0.56611479, -0.75179025, -1.62742151, ...,  0.85971666,
          1.41735034, -0.01672347],
        [-3.18950874, -3.59072278, -3.4365621 , ..., -1.04086233,
         -0.83203235, -0.0456026 ],
        ...,
        [-2.31504409, -1.69810109, -1.62742151, ..., -1.55177066,
         -0.83203235, -0.03796454],
        [ 0.30834986,  0.19452059,  0.18171908, ..., -0.039482  ,
         -0.38215581, -0.03309073],
        [ 1.18281452,  1.14083144,  1.08628938, ...,  0.85971666,
         -0.83203235, -0.03698251]]),
 array([[-2.31504409,  1.14083144, -2.5319918 , ..., -0.77519   ,
         -0.38215581, -0.03756445],
        [ 0.30834986,  1.14083144,  0.18171908, ..., -1.47002532,
         -0.38215581, -0.03230874],
        [ 0.30834986, -3.59072278,  1.08628938, ...,  0.85971666,
         -0.83203235, -0.03458197],
        ...,
        [-2.31504409,  1.14083144, -0.72285121, ...,  

In [12]:
# Модель линейной регрессии
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

X_train, X_test, y_train, y_test = split_scale (data)
# Модель
lrg = LinearRegression()
lrg.fit(X_train, y_train)
y_pred = lrg.predict(X_test)

# Метрики
r2 = r2_score(y_test, y_pred)
print('R2: ', r2)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE : %.3f' % rmse)

R2:  -0.00012798006789505045
Test RMSE : 19374687.467


Заметим что значение коэффициента детерминации (R2) отрицательно. Это говорит о крайней неадекватности модели. 

Применим методы снижения размерности для повышения качества модели.

МЕТОДЫ СНИЖАНИЯ РАЗМЕРНОСТИ

1. Метод главных компонент. 

Данный метод аппроксимирует n-размерное облако наблюдений до эллипсоида (тоже n-мерного), полуоси которого и будут являться будущими главными компонентами. И при проекции на такие оси (снижении размерности) сохраняется наибольшее количество информации.

In [38]:
# Метод главных компонент
arr_for_x = []
for i in range(len(data.columns)):
        arr_for_x.append(i)
y = arr_for_x.pop(165)
#     y = arr_for_x.pop(168)
X, y = d= data.iloc[:,arr_for_x],  data.iloc[:,y]

from sklearn.decomposition import PCA

pca = PCA(.99) # Сохраняем информативность 95% (дисперсия)
X_centered = X - X.mean()
pca.fit(X_centered)
X_pca = pca.transform(X_centered)
# pca.fit(X) # Обучаем только на тренирововчной выборке
# print('Chosen ', pca.n_components_, ' components from 168' )
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.25, random_state=12)


# X_train = pca.transform(X_train) # Перестраиваем тестовую и тренировочную
# X_test = pca.transform(X_te

# print(pca.explained_variance_ratio_)
print('полученная информативность: ',np.sum(pca.explained_variance_ratio_))

полученная информативность:  0.9999724614590487


In [39]:
# Применяем модель линейной регрессии для выбранных параметров
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import SGDRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn import tree
# from sklearn.linear_model import Lasso
# from sklearn.model_selection import cross_val_score
# from sklearn.ensemble import GradientBoostingRegressor
# lrg = GradientBoostingRegressor()
# from sklearn.ensemble import RandomForestRegressor
# lrg = RandomForestRegressor()
# model_ordinal = OrdinalRidge()
# lrg = Lasso()
# lrg = tree.DecisionTreeRegressor()
# lrg = KNeighborsRegressor()
# lrg = SGDRegressor()
lrg = LinearRegression()
lrg.fit(X_train, y_train)
y_pred = lrg.predict(X_test)

# Метрики
# print(cross_val_score(lrg, X_train))
r2_1 = r2_score(y_test, y_pred)
print('R2 (1): ', r2_1)
rmse1 = sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE (1): %.3f' % rmse1)

R2 (1):  -7.666536062811424e-05
Test RMSE (1): 19374190.421


Методы:

2 - Singular value decomposition

3 - Non-negative matrix factorization

4 - ICA

Метрики для методов????